In [31]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV


#Reading the data

In [32]:
house_df = pd.read_csv("/Users/conniechou/Downloads/AmesHousing.csv")
house_df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


# Model 1: using only size and number of rooms

In [33]:
#size = GrLivArea
#number of rooms = TotRms AbvGrd
X = house_df.drop("SalePrice", axis = 1) #keep all columns except ur y
y = house_df["SalePrice"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [34]:
lr = LinearRegression()
std_s = StandardScaler()

ct = ColumnTransformer(
    [("standardize", std_s, ["Gr Liv Area", "TotRms AbvGrd"])],
    remainder = "drop" #drop the columns we dont need
)

lr_pipeline = Pipeline(
  [("standardize", ct),
  ("linear_regression", lr)]
)

lr_pipeline_fitted = lr_pipeline.fit(X_train, y_train) #fit the model with training data (learn from training data)
y_preds = lr_pipeline_fitted.predict(X_test)

In [35]:
rmse1 = mean_squared_error(y_test, y_preds, squared = False)
rmse1

51760.39520451284

#Model 2: Using size, number of rooms, and building type

In [36]:
enc = OneHotEncoder(sparse_output = False)

ct = ColumnTransformer(
    [("standardize", std_s, ["Gr Liv Area", "TotRms AbvGrd"]),
     ("dummify", enc, ["Bldg Type"])],
    remainder = "drop" #drop the columns we dont need
)

lr_pipeline2 = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", lr)]
)

lr_pipeline_fitted2 = lr_pipeline2.fit(X_train, y_train) #fit the model with training data (learn from training data)
y_preds2 = lr_pipeline_fitted2.predict(X_test)

In [37]:
rmse2 = mean_squared_error(y_test, y_preds2, squared = False)
rmse2

50476.8244154964

#Model 3: using size and building type, and their interaction

In [38]:
#size = numerical
#building type = dummy
enc = OneHotEncoder(sparse_output = False)

#first transform building type into dummy in its own column transformer
ct_dummy = ColumnTransformer(
    [("dummify", enc, ["Bldg Type"])], remainder = "passthrough"
).set_output(transform = "pandas")

ct_dummy.fit_transform(X_train)

#then another column transformer to have interaction between size and dummified building type
ct_interact = ColumnTransformer(
    [("interaction", PolynomialFeatures(interaction_only = True), ["remainder__Gr Liv Area", "dummify__Bldg Type_1Fam"]),
    ("interaction2", PolynomialFeatures(interaction_only = True), ["remainder__Gr Liv Area", "dummify__Bldg Type_2fmCon"]),
    ("interaction3", PolynomialFeatures(interaction_only = True), ["remainder__Gr Liv Area", "dummify__Bldg Type_Duplex"]),
    ("interaction4", PolynomialFeatures(interaction_only = True), ["remainder__Gr Liv Area", "dummify__Bldg Type_Twnhs"]),
    ("interaction5", PolynomialFeatures(interaction_only = True), ["remainder__Gr Liv Area", "dummify__Bldg Type_TwnhsE"])],
).set_output(transform = "pandas")

lr_pipeline_inter = Pipeline(
    [("dummify", ct_dummy),
     ("interacting", ct_interact),
      ("linear regression", lr)]
).set_output(transform = "pandas")

#fitting model on training data
lr_pipeline_fitted3 = lr_pipeline_inter.fit(X_train, y_train) #fit the model with training data (learn from training data)
y_preds3 = lr_pipeline_fitted3.predict(X_test)

In [39]:
rmse3 = mean_squared_error(y_test, y_preds3, squared = False)
rmse3

50192.90700604007

#Model 4: using a 5-degree polynomial on size, a 5 degree polynomial on number of rooms, and also building type

In [40]:
#size = numerical
#building type = dummy
enc = OneHotEncoder(sparse_output = False)

#first transform building type into dummy in its own column transformer
ct_preprocess = ColumnTransformer(
    [("dummify", enc, ["Bldg Type"]),
     ("polynomial", PolynomialFeatures(degree = 5), ["Gr Liv Area", "TotRms AbvGrd"])], remainder = "drop"
).set_output(transform = "pandas")

ct_preprocess.fit_transform(X_train)

lr_pipeline_poly = Pipeline(
    [("preprocessing", ct_preprocess),
      ("linear regression", lr)]
).set_output(transform = "pandas")

#fitting model on training data
lr_pipeline_fitted4 = lr_pipeline_poly.fit(X_train, y_train) #fit the model with training data (learn from training data)
y_preds4 = lr_pipeline_fitted4.predict(X_test)

In [41]:
rmse4 = mean_squared_error(y_test, y_preds4, squared = False)
rmse4

120519.68755706924

The model that performed the best (that had the lowest RSME) is model 2.

#Cross Validation

In [42]:
#rsme cross validation for model 1
scores = cross_val_score(lr_pipeline, X, y, cv=5, scoring='neg_root_mean_squared_error')
-scores.mean()

55806.32634926364

In [43]:
#rsme cross validation for model 2
scores2 = cross_val_score(lr_pipeline2, X, y, cv=5, scoring='neg_root_mean_squared_error')
-scores2.mean()

54168.08142919383

In [44]:
#rsme cross validation for model 3
scores3 = cross_val_score(lr_pipeline_inter, X, y, cv=5, scoring='neg_root_mean_squared_error')
-scores3.mean()

53430.92197532802

In [45]:
#rsme cross validation for model 4
scores4 = cross_val_score(lr_pipeline_poly, X, y, cv=5, scoring='neg_root_mean_squared_error')
-scores4.mean()

61241.88398053136

TO INTERPRET RSME, TAKE THE ABSOLUTE VALUE
After taking the absolute value, we can see that model 3 performed better.

# Tuning

In [46]:
ct_tuning = ColumnTransformer(
  [("size_polynomial", PolynomialFeatures(), ["Gr Liv Area"]),
  ("rooms_polynomial", PolynomialFeatures(), ["TotRms AbvGrd"]),
   ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])],
  remainder = "drop"
)

lr_pipeline_tuning = Pipeline(
  [("preprocessing", ct_tuning),
  ("linear_regression", lr)]
).set_output(transform="pandas")

degrees = {'preprocessing__size_polynomial__degree': np.arange(1, 11),
           "preprocessing__rooms_polynomial__degree": np.arange(1, 11)} #create a dictionary of the polynomials

gscv = GridSearchCV(lr_pipeline_tuning, degrees, cv = 5, scoring='r2')

gscv_fitted = gscv.fit(X, y)

gscv_fitted.cv_results_ #ugly format, so put it into a pandas dataframe for easy reading

{'mean_fit_time': array([0.01782379, 0.01368637, 0.01301761, 0.01314769, 0.01346421,
        0.0134912 , 0.01357036, 0.01354413, 0.01465802, 0.0150774 ,
        0.01293478, 0.01296763, 0.01298542, 0.01353526, 0.0134367 ,
        0.01359711, 0.01348009, 0.01366663, 0.01381025, 0.01419125,
        0.01307683, 0.01302123, 0.01344252, 0.01342287, 0.01362429,
        0.01366515, 0.01362715, 0.01379156, 0.01387644, 0.01385956,
        0.01347752, 0.01380897, 0.01431241, 0.01366525, 0.01351953,
        0.01369233, 0.01636329, 0.01396837, 0.0142899 , 0.01468596,
        0.01345592, 0.01319957, 0.01358786, 0.01375957, 0.01386685,
        0.0141479 , 0.01376929, 0.01459794, 0.01447196, 0.01440783,
        0.01322613, 0.01342053, 0.01346073, 0.01398301, 0.01366448,
        0.0152092 , 0.01592655, 0.01914101, 0.01475201, 0.0154304 ,
        0.01325259, 0.01352353, 0.01358781, 0.01377397, 0.01405635,
        0.01439557, 0.01427712, 0.01450505, 0.01601872, 0.01928225,
        0.02296472, 0.02090893,

In [47]:
gscv_df = pd.DataFrame(gscv_fitted.cv_results_)
gscv_df = gscv_df[["param_preprocessing__rooms_polynomial__degree",
                   "param_preprocessing__size_polynomial__degree",
                   "mean_test_score"]]
gscv_df.rename(columns = {
    "param_preprocessing__rooms_polynomial__degree" : "room_degrees",
    "param_preprocessing__size_polynomial__degree" : "size_degrees",
    "mean_test_score": "r2"
}, inplace = True)

gscv_df.sort_values(by = "r2", ascending = False).head()

,room_degrees,size_degrees,r2
2,1,3,0.557641
12,2,3,0.556857
33,4,4,0.556855
43,5,4,0.556531
22,3,3,0.554039



The best final model is the model with the polynomial degree of 3 for the size of the house and a polynomial degree of 1 for the number of rooms. The R^2 value is 0.557641.